# Create an AI Agent with Langchain

## Intall and Import Libraries

In [ ]:
%pip install --upgrade --quiet sqlalchemy langchain langchain-community langchain-openai

Note: We use Google Colab's Secrets feature to store and retreive sensitive information like API-Keys.

Get your OpenAI API-Key: https://platform.openai.com/

## Load CSV Data

I used an example .csv data stored in the GitHub repo of the project: https://github.com/OktayBogazkaya/ai-trading-journal/tree/main/data

Feel free to download and use the dataset, or use your own.


In [ ]:
import pandas as pd

# Load CSV into pandas
df = pd.read_csv("trading_data.csv")
df.head(5)

,Date,Ticker,Quantity,Entry Price ($),Exit Price ($),Commission ($),Profit/Loss ($)
0,2023-01-15,IWM,352,20.93,30.68,5.0,3427.00
1,2023-07-27,TSM,205,221.50,217.45,5.0,-835.25
2,2023-06-02,INTC,171,130.64,137.52,5.0,1171.48
3,2023-10-09,AAPL,12,413.08,404.85,5.0,-103.76
4,2023-06-18,QQQ,149,35.30,45.29,5.0,1483.51


## Load CSV into SQLite Database

In [ ]:
import sqlite3
from sqlalchemy import create_engine
# Create an SQLite database engine with SQLAlchemy
engine = create_engine("sqlite:///trading_journal.db")

df.to_sql("trades", engine, if_exists="replace", index=False)

500

## Create Agent to interact with SQL DB

---



In [ ]:
from google.colab import userdata
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent
from langchain.chat_models import ChatOpenAI

# Connect Langchain agent with SQL database
db = SQLDatabase(engine=engine)

#Instantiate the LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0, openai_api_key=userdata.get('OPENAI_API_KEY'))

# Create Agent
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=False)

## Chat with your Data

In [ ]:
response = agent_executor.invoke({"input": "What is the total number of trades in 2023"})

print(response["output"])

In [ ]:
# Cross check
len(df["Date"])

In [ ]:
response = agent_executor.invoke({"input": "What was my highest winning trade on Meta (Ticker: FB) in 2023?"})

print(response["output"])


In [ ]:
# Cross check
print(df[df["Ticker"] == "FB"]["Profit/Loss ($)"].max())

3637.03


In [ ]:
# Create Agent with verbose=True
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

response = agent_executor.invoke({"input": "What was my highest winning trade on Meta (Ticker: FB) in 2023?"})

print(response["output"])



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


trades
Invoking: `sql_db_schema` with `{'table_names': 'trades'}`



CREATE TABLE trades (
	"Date" TEXT, 
	"Ticker" TEXT, 
	"Quantity" BIGINT, 
	"Entry Price ($)" FLOAT, 
	"Exit Price ($)" FLOAT, 
	"Commission ($)" FLOAT, 
	"Profit/Loss ($)" FLOAT
)

/*
3 rows from trades table:
Date	Ticker	Quantity	Entry Price ($)	Exit Price ($)	Commission ($)	Profit/Loss ($)
2023-01-15	IWM	352	20.93	30.68	5.0	3427.0
2023-07-27	TSM	205	221.5	217.45	5.0	-835.25
2023-06-02	INTC	171	130.64	137.52	5.0	1171.48
*/
Invoking: `sql_db_query_checker` with `{'query': 'SELECT MAX("Profit/Loss ($)") AS MaxProfit FROM trades WHERE Ticker = \'FB\' AND strftime(\'%Y\', Date) = \'2023\''}`


```sql
SELECT MAX("Profit/Loss ($)") AS MaxProfit FROM trades WHERE Ticker = 'FB' AND strftime('%Y', Date) = '2023'
```
Invoking: `sql_db_query` with `{'query': 'SELECT MAX("Profit/Loss ($)") AS MaxProfit FROM trades WHERE Ticker = \'FB\' AND 

## Note
I named the notebook and Github repo **AI Trading Journal**, because this is just the beginning of a much larger project.

More to come soon! Stay tuned and follow RhinoAcademy 👇

- [Youtube Channel](https://www.youtube.com/@Rhino-Academy)
- [Substack](https://rhinoacademy.substack.com/)
- [GitHub Repo](https://github.com/OktayBogazkaya/ai-trading-journal)